In [9]:
import os
import dotenv
import fnmatch
import requests
import csv

import pandas as pd 

from langchain_google_genai import GoogleGenerativeAI

from bs4 import BeautifulSoup

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [32]:
dotenv.load_dotenv()
google_api_key = os.getenv('google_api_key')

urls = ['https://uww.org/event/seniors-pan-american-championships/results']

url = urls[0]

In [3]:
import google.generativeai as genai

genai.configure(api_key=google_api_key)
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


El modelo 'models/gemini-1.5-pro' será usado ya que está optimizado para tareas de razonamiento complejas, como la generación de código y texto, la edición de texto, la resolución de problemas y la extracción y generación de datos

In [5]:
model = GoogleGenerativeAI(
  model='models/gemini-1.5-pro',
  google_api_key=google_api_key
)

In [41]:
response = requests.get(url)
if response.status_code == 200:
  soup = BeautifulSoup(response.content, 'html.parser')
  body = soup.body
  print(body)
  with open('index.html', 'w', encoding='utf-8') as file:
    file.write(body)
else:
  print("Error al acceder a {}: {}".format(url, response.status_code))


In [ ]:
with open('index_mod.html', 'r', encoding='utf-8') as file:
  html_content = file.read()

#chain = prompt_template_html_to_csv()
#result = chain.invoke({'html_str':html_content})
#print(result)

Atleta 1, Atleta 2, Resultado Atleta 1, Resultado Atleta 2, Ganador, Etapa,
Spencer Richard LEE, Pedro Jesus MEJIAS RODRIGUEZ, 12, 2, Spencer Richard LEE, Gold,
Oscar Eduardo TIGREROS URBANO, Davi SILVA GIOVANNETTI, 13, 2, Oscar Eduardo TIGREROS URBANO, Bronze,
Guesseppe Ricardo REA VILLARROEL, Jaime Isaac PEREZ CASTELLANOS, - , - , Guesseppe Ricardo REA VILLARROEL, Bronze,
Pedro Jesus MEJIAS RODRIGUEZ, Guesseppe Ricardo REA VILLARROEL, - , - , Pedro Jesus MEJIAS RODRIGUEZ, Semifinal,
Spencer Richard LEE, Davi SILVA GIOVANNETTI, 10, 0, Spencer Richard LEE, Semifinal,
Spencer Richard LEE, Oscar Eduardo TIGREROS URBANO, 10, 0, Spencer Richard LEE, Quarterfinal,
Davi SILVA GIOVANNETTI, Jorge Alberto OLVERA RODRIGUEZ, 9, 9, Davi SILVA GIOVANNETTI, Quarterfinal,
Pedro Jesus MEJIAS RODRIGUEZ, Jaime Isaac PEREZ CASTELLANOS, 11, 0, Pedro Jesus MEJIAS RODRIGUEZ, Quarterfinal,
Guesseppe Ricardo REA VILLARROEL, Peter Lewis HAMMER CUDE, 13, 11, Guesseppe Ricardo REA VILLARROEL, Quarterfinal,
Oscar

In [35]:
print(soup.prettify())

<!DOCTYPE html>
<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D%7D" data-n-head-ssr="" lang="en">
 <head>
  <meta charset="utf-8" data-n-head="ssr"/>
  <meta content="ie=edge" data-n-head="ssr" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1,maximum-scale=1" data-n-head="ssr" name="viewport"/>
  <meta content="#da532c" data-n-head="ssr" name="msapplication-TileColor"/>
  <meta content="/static-assets/favicons/browserconfig.xml" data-n-head="ssr" name="msapplication-config"/>
  <meta content="#ffffff" data-n-head="ssr" name="theme-color"/>
  <meta content="" data-n-head="ssr" property="fb:pages"/>
  <meta content="1098452874427866" data-n-head="ssr" property="fb:app_id"/>
  <meta content="Get the UWW Seniors Pan-American Championships 2024 wrestling results and scores for the men's Greco-Roman and men and women's freestyle categories across all weight classes." data-n-head="ssr" name="description"/>
  <meta content="" data-n-head="ssr" prope

In [25]:
template = """
Del archivo siguiente, extraeme: Atleta 1, Atleta 2, Resultado, Ganador, Etapa

{html_str}

Ejemplo 1:

<h3 class="tabs-container-title">Gold</h3>
<div class="card-data"><div class="">Spencer Richard LEE</div> 
<div class="card-number">12</div>
<div class="card-data"><div class="">Pedro Jesus MEJIAS RODRIGUEZ</div>
<div class="card-number">2</div>

Respuesta de Ejemplo 1:
Atleta 1: Spencer Richard LEE
Atleta 2: Pedro Jesus MEJIAS RODRIGUEZ
Resultado Atleta 1: 12
Resultado Atleta 2: 2
Ganador: Spencer Richard LEE
Etapa: Gold

Ejemplo 2:
<h3 class="tabs-container-title">Bronze</h3>
<div class="card-item card-a won">
<div class="card-data"><div class="">Oscar Eduardo TIGREROS URBANO</div>
<div class="card-number">13</div>
<div class="card-data"><div class="">Davi SILVA GIOVANNETTI</div>
<div class="card-number">2</div>

Respuesta de Ejemplo 2:
Atleta 1: Eduardo TIGREROS URBANO
Atleta 2: Davi SILVA GIOVANNETTI
Resultado Atleta 1: 13
Resultado Atleta 2: 2
Ganador: Eduardo TIGREROS URBANO
Etapa: Bronze


Respuesta que tienes que devolvermela de la siguiente forma:

Atleta 1, Atleta 2, Resultado Atleta 1, Resultado Atleta 2, Ganador, Etapa,
Spencer Richard LEE, Pedro Jesus MEJIAS RODRIGUEZ, 12, 2, Spencer Richard LEE, Gold
Eduardo TIGREROS URBANO, Davi SILVA GIOVANNETTI, 13, 2, Eduardo TIGREROS URBANO, Bronze
"""

def prompt_template_html_to_csv():
  prompt = ChatPromptTemplate.from_template(template)
  chain = prompt|model
  return chain

In [26]:
with open('index_mod.html', 'r', encoding='utf-8') as file:
  html_content = file.read()

chain = prompt_template_html_to_csv()
result = chain.invoke({'html_str':html_content})
print(result)

Atleta 1, Atleta 2, Resultado Atleta 1, Resultado Atleta 2, Ganador, Etapa,
Spencer Richard LEE, Pedro Jesus MEJIAS RODRIGUEZ, 12, 2, Spencer Richard LEE, Gold,
Oscar Eduardo TIGREROS URBANO, Davi SILVA GIOVANNETTI, 13, 2, Oscar Eduardo TIGREROS URBANO, Bronze,
Guesseppe Ricardo REA VILLARROEL, Jaime Isaac PEREZ CASTELLANOS, - , - , Guesseppe Ricardo REA VILLARROEL, Bronze,
Pedro Jesus MEJIAS RODRIGUEZ, Guesseppe Ricardo REA VILLARROEL, - , - , Pedro Jesus MEJIAS RODRIGUEZ, Semifinal,
Spencer Richard LEE, Davi SILVA GIOVANNETTI, 10, 0, Spencer Richard LEE, Semifinal,
Spencer Richard LEE, Oscar Eduardo TIGREROS URBANO, 10, 0, Spencer Richard LEE, Quarterfinal,
Davi SILVA GIOVANNETTI, Jorge Alberto OLVERA RODRIGUEZ, 9, 9, Davi SILVA GIOVANNETTI, Quarterfinal,
Pedro Jesus MEJIAS RODRIGUEZ, Jaime Isaac PEREZ CASTELLANOS, 11, 0, Pedro Jesus MEJIAS RODRIGUEZ, Quarterfinal,
Guesseppe Ricardo REA VILLARROEL, Peter Lewis HAMMER CUDE, 13, 11, Guesseppe Ricardo REA VILLARROEL, Quarterfinal,
Oscar

In [29]:
file_name = 'testing1'
with open(f'{file_name}.csv', 'w', encoding='utf-8') as csv_file:
  writer = csv.writer(csv_file)
  for content in result.split('\n'):
    if not content.strip():
      continue
    writer.writerow(content.split(','))